In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy
import pandas as pd

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract pillow tqdm torch
import pytesseract
from datasets import Dataset, Features, Value
from transformers import AutoProcessor
from tqdm import tqdm
import torch

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [ ]:
from datasets import load_from_disk

# Reload your processed data
encoded_dataset = {
    'train': load_from_disk("/content/drive/MyDrive/encoded_data/train"),
    'val': load_from_disk("/content/drive/MyDrive/encoded_data/val"),
    'test': load_from_disk("/content/drive/MyDrive/encoded_data/test")
}

print("✅ Encoded dataset reloaded!")

✅ Encoded dataset reloaded!


In [ ]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-g71131ul
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-g71131ul
  Resolved https://github.com/facebookresearch/detectron2.git to commit a1ce2f956a1d2212ad672e3c47d53405c2fe4312
  Preparing metadata (setup.py) ... done


In [ ]:
from transformers import LayoutLMv2ForSequenceClassification

# Initialize model for classification
model = LayoutLMv2ForSequenceClassification.from_pretrained(
    "microsoft/layoutlmv2-base-uncased",
    num_labels=6   # Your 6 document classes
)

print("✅ Model initialized!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/802M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/802M [00:00<?, ?B/s]

Some weights of LayoutLMv2ForSequenceClassification were not initialized from the model checkpoint at microsoft/layoutlmv2-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Model initialized!


In [ ]:
for split_name in encoded_dataset:
    encoded_dataset[split_name].set_format(type='torch')

print("✅ Dataset converted to PyTorch format")

✅ Dataset converted to PyTorch format


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./documind-results",
    num_train_epochs=3,
    per_device_train_batch_size=4,  # Reduced for memory
    per_device_eval_batch_size=4,
    learning_rate=1e-4,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    remove_unused_columns=False,  # Important!
    dataloader_pin_memory=False,  # Important for Colab
    report_to=None,  # Disable wandb if not needed
)

print("✅ Training arguments configured")

✅ Training arguments configured


In [ ]:
# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["val"],
)

# START TRAINING! 🎯
print("🚀 Starting training...")
trainer.train()

🚀 Starting training...


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sahil-20222044 (sahil-20222044-motilal-nehru-national-institute-of-techn) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,0.973005
2,1.166100,0.758413
3,0.727400,0.633217


TrainOutput(global_step=1350, training_loss=0.8393538298430266, metrics={'train_runtime': 2062.6, 'train_samples_per_second': 2.618, 'train_steps_per_second': 0.655, 'total_flos': 2915247332966400.0, 'train_loss': 0.8393538298430266, 'epoch': 3.0})

In [ ]:
model.save_pretrained("/content/drive/MyDrive/documind-model")

In [ ]:
eval_results = trainer.evaluate()
print(f"📊 Validation Results: {eval_results}")

# Test on unseen test set
test_results = trainer.evaluate(encoded_dataset["test"])
print(f"📊 Test Results: {test_results}")

📊 Validation Results: {'eval_loss': 0.6332173943519592, 'eval_runtime': 87.1109, 'eval_samples_per_second': 6.888, 'eval_steps_per_second': 1.722, 'epoch': 3.0}
📊 Test Results: {'eval_loss': 0.5766794085502625, 'eval_runtime': 81.6393, 'eval_samples_per_second': 7.349, 'eval_steps_per_second': 1.837, 'epoch': 3.0}


In [ ]:
trainer.save_model("/content/drive/MyDrive/documind-model")

In [ ]:
import re
import dateutil.parser
from typing import Dict, List, Optional, Any
from datetime import datetime
import json


def parse_date(date_str: str) -> Optional[str]:
    """Parse various date formats to standard format"""
    try:
        parsed = dateutil.parser.parse(date_str, fuzzy=True)
        return parsed.strftime('%Y-%m-%d')
    except:
        return None

def extract_currency(text: str) -> Optional[str]:
    """Extract currency information"""
    currencies = ['USD', 'EUR', 'GBP', 'INR', 'CAD', 'AUD']
    for currency in currencies:
        if currency in text:
            return currency

    if '$' in text:
        return 'USD'
    elif '€' in text:
        return 'EUR'
    elif '£' in text:
        return 'GBP'

    return None

def is_line_item(line: str) -> bool:
    """Check if line looks like a line item"""
    has_numbers = bool(re.search(r'\d', line))
    has_price = bool(re.search(r'\$?\d+\.?\d*', line))
    reasonable_length = 5 < len(line.strip()) < 200
    return has_numbers and has_price and reasonable_length

def parse_line_item(line: str) -> Optional[Dict]:
    """Parse a line item"""
    patterns = [
        r'(\d+)\s+(.+?)\s+\$?(\d+\.?\d*)\s+\$?(\d+\.?\d*)',
        r'(.+?)\s+\$?(\d+\.?\d*)',
    ]

    for pattern in patterns:
        match = re.search(pattern, line)
        if match:
            if len(match.groups()) == 4:
                return {
                    'quantity': int(match.group(1)),
                    'description': match.group(2).strip(),
                    'unit_price': float(match.group(3)),
                    'total_price': float(match.group(4))
                }
            elif len(match.groups()) == 2:
                return {
                    'description': match.group(1).strip(),
                    'amount': float(match.group(2))
                }
    return None

def extract_email_addresses(text: str) -> List[str]:
    """Extract email addresses"""
    return re.findall(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', text)

def extract_phone_numbers(text: str) -> List[str]:
    """Extract phone numbers"""
    phone_pattern = r'(\+?\d{1,3}[-.\s]?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}'
    return re.findall(phone_pattern, text)

def extract_amounts(text: str) -> List[float]:
    """Extract monetary amounts"""
    amount_pattern = r'\$?(\d{1,3}(?:,\d{3})*\.?\d{0,2})'
    amounts = re.findall(amount_pattern, text)

    numeric_amounts = []
    for amt in amounts:
        try:
            clean_amt = float(amt.replace(',', ''))
            if 0.01 <= clean_amt <= 10000000:
                numeric_amounts.append(clean_amt)
        except:
            continue

    return numeric_amounts

def extract_dates(text: str) -> List[str]:
    """Extract all dates from text"""
    date_pattern = r'\b(\d{1,2}[/-]\d{1,2}[/-]\d{2,4}|\d{4}-\d{2}-\d{2}|\w+ \d{1,2},? \d{4})\b'
    dates = re.findall(date_pattern, text)

    parsed_dates = []
    for date_str in dates:
        parsed = parse_date(date_str)
        if parsed:
            parsed_dates.append(parsed)

    return parsed_dates

In [ ]:
class StructuredDataExtractor:
    def __init__(self):
        self.setup_extractors()

    def setup_extractors(self):
        """Initialize all document type extractors"""
        self.invoice_extractor = InvoiceExtractor()
        self.resume_extractor = ResumeExtractor()
        self.news_extractor = NewsArticleExtractor()
        self.email_extractor = EmailExtractor()
        self.letter_extractor = LetterExtractor()
        self.form_extractor = FormExtractor()

    def extract_structured_data(self, doc_type: str, text: str, layout_data: Dict = None) -> Dict[str, Any]:
        """Extract structured data based on document type"""
        if layout_data is None:
            layout_data = {}

        extractors = {
            'invoice': self.invoice_extractor.extract_invoice_data,
            'resume': self.resume_extractor.extract_resume_data,
            'news_article': self.news_extractor.extract_news_data,
            'email': self.email_extractor.extract_email_data,
            'letter': self.letter_extractor.extract_letter_data,
            'form': self.form_extractor.extract_form_data
        }

        if doc_type in extractors:
            return extractors[doc_type](text, layout_data)
        else:
            return self._extract_generic_data(text)

    def _extract_generic_data(self, text: str) -> Dict:
        """Extract generic data for unknown document types"""
        return {
            'text_preview': text[:500],
            'word_count': len(text.split()),
            'dates_found': extract_dates(text),
            'emails_found': extract_email_addresses(text),
            'phones_found': extract_phone_numbers(text)
        }

In [ ]:
class InvoiceExtractor:
    def extract_invoice_data(self, text: str, layout_data: Dict) -> Dict:
        lines = text.split('\n')

        return {
            'vendor': self._extract_vendor(lines),
            'invoice_number': self._extract_invoice_number(text),
            'invoice_date': self._extract_invoice_date(text),
            'due_date': self._extract_due_date(text),
            'total_amount': self._extract_total_amount(text),
            'tax_amount': self._extract_tax_amount(text),
            'line_items': self._extract_line_items(text, lines),
            'payment_terms': self._extract_payment_terms(text),
            'currency': extract_currency(text),  # USING SHARED HELPER
            'vendor_address': self._extract_vendor_address(lines),
            'client_info': self._extract_client_info(text)
        }

    def _extract_vendor(self, lines: List[str]) -> Optional[str]:
        patterns = [r'from:\s*(.+)', r'vendor:\s*(.+)', r'company:\s*(.+)', r'^([A-Z][a-zA-Z\s&]+(?:Inc|LLC|Corp|Company))']
        for line in lines[:10]:
            for pattern in patterns:
                match = re.search(pattern, line, re.IGNORECASE)
                if match and len(match.group(1).strip()) > 3:
                    return match.group(1).strip()
        return None

    def _extract_invoice_number(self, text: str) -> Optional[str]:
        patterns = [r'invoice\s*#?\s*[:#]?\s*([A-Z0-9-]+)', r'inv\.?\s*(\w+)']
        for pattern in patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                return match.group(1)
        return None

    def _extract_invoice_date(self, text: str) -> Optional[str]:
        date_patterns = [
            r'invoice\s+date[:]?\s*([\d/.-]+)',
            r'date[:]?\s*([\d/.-]+)',
            r'(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})'
        ]
        for pattern in date_patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                return parse_date(match.group(1))  # USING SHARED HELPER
        return None

    def _extract_total_amount(self, text: str) -> Optional[float]:
        patterns = [r'total\s*[$]?\s*([\d,]+\.?\d*)', r'amount\s+due\s*[$]?\s*([\d,]+\.?\d*)', r'balance\s+due\s*[$]?\s*([\d,]+\.?\d*)']
        for pattern in patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                try:
                    return float(match.group(1).replace(',', ''))
                except:
                    continue

        # Fallback to shared amounts extraction
        amounts = extract_amounts(text)  # USING SHARED HELPER
        return max(amounts) if amounts else None

    def _extract_line_items(self, text: str, lines: List[str]) -> List[Dict]:
        items = []
        in_table = False

        for i, line in enumerate(lines):
            if any(word in line.lower() for word in ['item', 'description', 'qty', 'quantity', 'price', 'amount', 'total']):
                in_table = True
                continue

            if in_table and is_line_item(line):  # USING SHARED HELPER
                item = parse_line_item(line)  # USING SHARED HELPER
                if item:
                    items.append(item)

            if in_table and not line.strip():
                break

        return items[:15]

    def _extract_due_date(self, text: str) -> Optional[str]:
        patterns = [r'due\s+date[:]?\s*([\d/.-]+)', r'payment\s+due[:]?\s*([\d/.-]+)']
        for pattern in patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                return parse_date(match.group(1))  # USING SHARED HELPER
        return None

    def _extract_tax_amount(self, text: str) -> Optional[float]:
        patterns = [r'tax\s*[$]?\s*([\d,]+\.?\d*)', r'vat\s*[$]?\s*([\d,]+\.?\d*)', r'gst\s*[$]?\s*([\d,]+\.?\d*)']
        for pattern in patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                try:
                    return float(match.group(1).replace(',', ''))
                except:
                    continue
        return None

    def _extract_payment_terms(self, text: str) -> Optional[str]:
        patterns = [r'payment\s+terms[:]?\s*(.+)', r'terms[:]?\s*(.+)']
        for pattern in patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                return match.group(1).strip()
        return None

    def _extract_vendor_address(self, lines: List[str]) -> Optional[str]:
        address_lines = []
        collecting = False

        for line in lines:
            if any(word in line.lower() for word in ['address', 'street', 'avenue', 'road', 'city', 'state', 'zip']):
                collecting = True
                continue

            if collecting and line.strip():
                address_lines.append(line.strip())

            if collecting and not line.strip() and address_lines:
                break

        return ', '.join(address_lines) if address_lines else None

    def _extract_client_info(self, text: str) -> Dict:
        client_info = {}
        lines = text.split('\n')

        for i, line in enumerate(lines):
            if 'bill to' in line.lower() or 'client' in line.lower():
                # Collect next few lines for client info
                for j in range(i+1, min(i+6, len(lines))):
                    client_line = lines[j]
                    if client_line.strip():
                        if ':' in client_line:
                            parts = client_line.split(':', 1)
                            client_info[parts[0].strip().lower()] = parts[1].strip()

        return client_info

In [ ]:
class ResumeExtractor:
    def __init__(self):
        self.skill_categories = {
            'programming': ['python', 'java', 'javascript', 'c++', 'c#', 'ruby', 'go', 'rust', 'swift', 'php'],
            'data_science': ['tensorflow', 'pytorch', 'scikit-learn', 'pandas', 'numpy', 'sql', 'spark', 'hadoop', 'tableau'],
            'web_development': ['react', 'angular', 'vue', 'django', 'flask', 'node.js', 'express', 'html', 'css'],
            'devops': ['aws', 'docker', 'kubernetes', 'jenkins', 'terraform', 'ansible', 'git', 'ci/cd'],
            'soft_skills': ['leadership', 'communication', 'teamwork', 'problem-solving', 'project management']
        }

    def extract_resume_data(self, text: str, layout_data: Dict) -> Dict:
        lines = text.split('\n')

        return {
            'personal_info': {
                'name': self._extract_name(lines),
                'email': extract_email_addresses(text),  # USING SHARED HELPER
                'phone': extract_phone_numbers(text),    # USING SHARED HELPER
                'location': self._extract_location(lines)
            },
            'education': self._extract_education(text),
            'experience': self._extract_experience(text),
            'skills': self._extract_skills(text),
            'summary': self._extract_summary(text),
            'certifications': self._extract_certifications(text),
            'languages': self._extract_languages(text)
        }

    def _extract_name(self, lines: List[str]) -> Optional[str]:
        for line in lines[:5]:
            if re.match(r'^[A-Z][a-z]+(?:\s+[A-Z][a-z]+){1,2}$', line.strip()):
                return line.strip()
        return None

    def _extract_experience(self, text: str) -> List[Dict]:
        experience = []
        lines = text.split('\n')

        for i, line in enumerate(lines):
            if any(keyword in line.lower() for keyword in ['experience', 'work history', 'employment']):
                for j in range(i+1, min(i+20, len(lines))):
                    exp_line = lines[j]
                    if self._is_experience_entry(exp_line):
                        exp = self._parse_experience_entry(exp_line)
                        if exp:
                            experience.append(exp)

        return experience[:10]

    def _is_experience_entry(self, line: str) -> bool:
        """Check if line looks like an experience entry"""
        has_dates = bool(re.search(r'\d{4}[-–]\d{4}|present|current', line.lower()))
        has_company = any(word in line.lower() for word in ['inc', 'corp', 'ltd', 'company', 'llc'])
        return has_dates or has_company

    def _parse_experience_entry(self, line: str) -> Optional[Dict]:
        """Parse experience entry"""
        # Simple parsing - can be enhanced
        date_match = re.search(r'(\d{4}[-–]\d{4}|present|current)', line)
        return {
            'position': line.strip(),
            'duration': date_match.group(1) if date_match else None,
            'company': self._extract_company_name(line)
        }

    def _extract_company_name(self, line: str) -> Optional[str]:
        """Extract company name from experience line"""
        # Simple extraction - can be enhanced
        words = line.split()
        for i, word in enumerate(words):
            if word.lower() in ['at', 'for', 'with'] and i + 1 < len(words):
                return ' '.join(words[i+1:])
        return None

    def _extract_education(self, text: str) -> List[Dict]:
        education = []
        lines = text.split('\n')

        for i, line in enumerate(lines):
            if any(keyword in line.lower() for keyword in ['education', 'university', 'college', 'bachelor', 'master', 'phd']):
                for j in range(i+1, min(i+6, len(lines))):
                    edu_line = lines[j]
                    if self._is_education_entry(edu_line):
                        education.append({'institution': edu_line.strip()})

        return education

    def _is_education_entry(self, line: str) -> bool:
        """Check if line looks like an education entry"""
        return len(line.strip()) > 5 and any(word in line.lower() for word in ['university', 'college', 'institute', 'school'])

    def _extract_skills(self, text: str) -> Dict:
        found_skills = {}
        text_lower = text.lower()

        for category, skills in self.skill_categories.items():
            found_skills[category] = [skill for skill in skills if skill in text_lower]

        return found_skills

    def _extract_location(self, lines: List[str]) -> Optional[str]:
        for line in lines[:10]:
            # Simple location pattern - city, state format
            if re.search(r'[A-Z][a-z]+,\s*[A-Z]{2}', line):
                return line.strip()
        return None

    def _extract_summary(self, text: str) -> Optional[str]:
        lines = text.split('\n')
        for i, line in enumerate(lines):
            if any(word in line.lower() for word in ['summary', 'objective', 'profile']):
                if i + 1 < len(lines):
                    return lines[i + 1].strip()
        return None

    def _extract_certifications(self, text: str) -> List[str]:
        certifications = []
        lines = text.split('\n')

        for i, line in enumerate(lines):
            if any(word in line.lower() for word in ['certification', 'certified', 'certificate']):
                certifications.append(line.strip())

        return certifications

    def _extract_languages(self, text: str) -> List[str]:
        languages = []
        lines = text.split('\n')

        for i, line in enumerate(lines):
            if 'languages' in line.lower():
                for j in range(i+1, min(i+4, len(lines))):
                    lang_line = lines[j]
                    if lang_line.strip():
                        languages.extend([lang.strip() for lang in lang_line.split(',')])
                break

        return languages

In [ ]:
class NewsArticleExtractor:
    def extract_news_data(self, text: str, layout_data: Dict) -> Dict:
        lines = text.split('\n')

        return {
            'headline': self._extract_headline(lines),
            'author': self._extract_author(text),
            'publication_date': self._extract_publication_date(text),
            'location': self._extract_location_news(text),
            'key_entities': self._extract_entities(text),
            'topics': self._extract_topics(text),
            'summary_points': self._extract_key_points(text),
            'quotes': self._extract_quotes(text),
            'statistics': self._extract_statistics(text)
        }

    def _extract_headline(self, lines: List[str]) -> Optional[str]:
        for line in lines[:3]:
            if len(line.strip()) > 10 and len(line.strip()) < 200:
                if line.strip().endswith('.'):
                    return line.strip()[:-1]
                return line.strip()
        return None

    def _extract_author(self, text: str) -> Optional[str]:
        patterns = [
            r'by\s+([A-Z][a-z]+\s+[A-Z][a-z]+)',
            r'author[:\s]+([A-Z][a-z]+\s+[A-Z][a-z]+)',
            r'written by\s+([A-Z][a-z]+\s+[A-Z][a-z]+)'
        ]
        for pattern in patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                return match.group(1)
        return None

    def _extract_publication_date(self, text: str) -> Optional[str]:
        patterns = [
            r'published[:]?\s*([\d/.-]+)',
            r'date[:]?\s*([\d/.-]+)',
        ]
        for pattern in patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                return parse_date(match.group(1))  # USING SHARED HELPER

        # Fallback to shared date extraction
        dates = extract_dates(text)  # USING SHARED HELPER
        return dates[0] if dates else None

    def _extract_location_news(self, text: str) -> Optional[str]:
        location_patterns = [
            r'in\s+([A-Z][a-z]+(?:,\s*[A-Z][a-z]+)*)',
            r'from\s+([A-Z][a-z]+(?:,\s*[A-Z][a-z]+)*)'
        ]
        for pattern in location_patterns:
            match = re.search(pattern, text)
            if match:
                return match.group(1)
        return None

    def _extract_entities(self, text: str) -> Dict:
        entities = {
            'people': re.findall(r'\b[A-Z][a-z]+\s+[A-Z][a-z]+\b', text),
            'organizations': re.findall(r'\b[A-Z][a-z]+\s+(?:Inc|Corp|Company|LLC)\b', text),
            'locations': re.findall(r'\b(?:[A-Z][a-z]+\s*)+,\s*[A-Z]{2}\b', text)
        }
        return entities

    def _extract_topics(self, text: str) -> List[str]:
        common_topics = ['politics', 'sports', 'technology', 'business', 'health', 'education', 'entertainment']
        found_topics = [topic for topic in common_topics if topic in text.lower()]
        return found_topics

    def _extract_key_points(self, text: str) -> List[str]:
        sentences = text.split('.')
        return [s.strip() for s in sentences[:3] if len(s.strip()) > 20]

    def _extract_quotes(self, text: str) -> List[str]:
        return re.findall(r'"[^"]+"', text)

    def _extract_statistics(self, text: str) -> List[str]:
        return re.findall(r'\b\d+%\b|\b\d+\.\d+%\b', text)

In [ ]:
class EmailExtractor:
    def extract_email_data(self, text: str, layout_data: Dict) -> Dict:
        lines = text.split('\n')

        return {
            'headers': {
                'from': self._extract_sender(lines),
                'to': self._extract_recipient(lines),
                'subject': self._extract_subject(lines),
                'date': self._extract_email_date(lines),
                'cc': self._extract_cc(lines)
            },
            'content': {
                'body': self._extract_email_body(lines),
                'greeting': self._extract_greeting(lines),
                'closing': self._extract_closing(lines),
                'signature': self._extract_signature(lines)
            },
            'attachments': self._extract_attachments(text),
            'action_items': self._extract_action_items(text),
            'urgency': self._extract_urgency(text)
        }

    def _extract_sender(self, lines: List[str]) -> Optional[str]:
        for i, line in enumerate(lines):
            if line.lower().startswith('from:'):
                return line[5:].strip()
        return None

    def _extract_recipient(self, lines: List[str]) -> Optional[str]:
        for i, line in enumerate(lines):
            if line.lower().startswith('to:'):
                return line[3:].strip()
        return None

    def _extract_subject(self, lines: List[str]) -> Optional[str]:
        for i, line in enumerate(lines):
            if line.lower().startswith('subject:'):
                return line[8:].strip()
        return None

    def _extract_email_date(self, lines: List[str]) -> Optional[str]:
        for i, line in enumerate(lines):
            if line.lower().startswith('date:'):
                return parse_date(line[5:].strip())  # USING SHARED HELPER
        return None

    def _extract_cc(self, lines: List[str]) -> List[str]:
        cc_emails = []
        for i, line in enumerate(lines):
            if line.lower().startswith('cc:'):
                emails = extract_email_addresses(line)  # USING SHARED HELPER
                cc_emails.extend(emails)
        return cc_emails

    def _extract_email_body(self, lines: List[str]) -> str:
        body_lines = []
        in_body = False

        for line in lines:
            if any(line.lower().startswith(header) for header in ['from:', 'to:', 'subject:', 'date:', 'cc:']):
                continue

            if line.strip() and not in_body:
                in_body = True

            if in_body:
                body_lines.append(line)

        return '\n'.join(body_lines).strip()

    def _extract_greeting(self, lines: List[str]) -> Optional[str]:
        for line in lines:
            if any(greeting in line.lower() for greeting in ['dear', 'hello', 'hi ', 'greetings']):
                return line.strip()
        return None

    def _extract_closing(self, lines: List[str]) -> Optional[str]:
        for line in reversed(lines):
            if any(closing in line.lower() for closing in ['sincerely', 'regards', 'best', 'thank you', 'thanks']):
                return line.strip()
        return None

    def _extract_signature(self, lines: List[str]) -> Optional[str]:
        for i, line in enumerate(lines):
            if any(sig in line.lower() for sig in ['signature', 'signed', 'regards']):
                if i + 1 < len(lines):
                    return lines[i + 1].strip()
        return None

    def _extract_attachments(self, text: str) -> List[str]:
        attachment_patterns = [
            r'attachment[:]?\s*(.+)',
            r'attached[:]?\s*(.+)',
            r'see\s+attached\s+(.+)'
        ]
        attachments = []
        for pattern in attachment_patterns:
            matches = re.findall(pattern, text, re.IGNORECASE)
            attachments.extend(matches)
        return attachments

    def _extract_action_items(self, text: str) -> List[str]:
        action_keywords = ['please', 'request', 'require', 'action', 'deadline', 'follow up', 'respond']
        sentences = text.split('.')
        action_sentences = [s.strip() for s in sentences if any(keyword in s.lower() for keyword in action_keywords)]
        return action_sentences[:5]

    def _extract_urgency(self, text: str) -> str:
        text_lower = text.lower()
        if any(word in text_lower for word in ['urgent', 'asap', 'immediately', 'emergency']):
            return 'high'
        elif any(word in text_lower for word in ['important', 'priority', 'soon']):
            return 'medium'
        else:
            return 'low'

In [ ]:
class LetterExtractor:
    def extract_letter_data(self, text: str, layout_data: Dict) -> Dict:
        lines = text.split('\n')

        return {
            'metadata': {
                'letter_type': self._detect_letter_type(text),
                'date': self._extract_letter_date(lines),
                'reference_number': self._extract_reference_number(text)
            },
            'parties': {
                'sender': self._extract_sender_info(lines),
                'recipient': self._extract_recipient_info(lines)
            },
            'content': {
                'subject': self._extract_letter_subject(lines),
                'purpose': self._extract_letter_purpose(text),
                'key_points': self._extract_key_points_letter(text),
                'action_required': self._extract_action_required(text)
            },
            'closing': {
                'signature': self._extract_letter_signature(lines),
                'attachments': self._extract_letter_attachments(text)
            }
        }

    def _detect_letter_type(self, text: str) -> str:
        text_lower = text.lower()
        if any(word in text_lower for word in ['job application', 'apply for', 'position of']):
            return 'job_application'
        elif any(word in text_lower for word in ['complaint', 'dissatisfied', 'issue with']):
            return 'complaint'
        elif any(word in text_lower for word in ['thank you', 'appreciation', 'grateful']):
            return 'thank_you'
        elif any(word in text_lower for word in ['resignation', 'leaving', 'last day']):
            return 'resignation'
        else:
            return 'business_correspondence'

    def _extract_letter_date(self, lines: List[str]) -> Optional[str]:
        for line in lines[:10]:
            date_match = re.search(r'(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})', line)
            if date_match:
                return parse_date(date_match.group(1))  # USING SHARED HELPER
        return None

    def _extract_reference_number(self, text: str) -> Optional[str]:
        patterns = [r'ref[:]?\s*([A-Z0-9-]+)', r'reference[:]?\s*([A-Z0-9-]+)']
        for pattern in patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                return match.group(1)
        return None

    def _extract_sender_info(self, lines: List[str]) -> Dict:
        sender_info = {}
        collecting = False

        for line in lines:
            if any(word in line.lower() for word in ['sincerely', 'yours truly', 'regards']):
                break

            if any(word in line.lower() for word in ['from:', 'sender:']):
                collecting = True
                continue

            if collecting and line.strip():
                if ':' in line:
                    parts = line.split(':', 1)
                    sender_info[parts[0].strip().lower()] = parts[1].strip()
                else:
                    sender_info['name'] = line.strip()

        return sender_info

    def _extract_recipient_info(self, lines: List[str]) -> Dict:
        recipient_info = {}
        collecting = False

        for line in lines:
            if 'dear' in line.lower():
                recipient_info['name'] = line.replace('Dear', '').replace('dear', '').strip().rstrip(',')
                collecting = True
                continue

            if collecting and line.strip():
                if ':' in line:
                    parts = line.split(':', 1)
                    recipient_info[parts[0].strip().lower()] = parts[1].strip()
                else:
                    if 'address' not in recipient_info:
                        recipient_info['address'] = line.strip()

        return recipient_info

    def _extract_letter_subject(self, lines: List[str]) -> Optional[str]:
        for line in lines:
            if line.lower().startswith('subject:'):
                return line[8:].strip()
            elif 're:' in line.lower():
                return line.strip()
        return None

    def _extract_letter_purpose(self, text: str) -> Optional[str]:
        sentences = text.split('.')
        for sentence in sentences[:3]:
            if len(sentence.strip()) > 20:
                return sentence.strip()
        return None

    def _extract_key_points_letter(self, text: str) -> List[str]:
        key_points = []
        sentences = text.split('.')

        action_words = ['request', 'require', 'suggest', 'recommend', 'propose', 'inform']
        for sentence in sentences:
            if any(word in sentence.lower() for word in action_words) and len(sentence.strip()) > 10:
                key_points.append(sentence.strip())

        return key_points[:5]

    def _extract_action_required(self, text: str) -> List[str]:
        action_items = []
        sentences = text.split('.')

        for sentence in sentences:
            if any(word in sentence.lower() for word in ['please', 'request', 'require', 'action needed', 'respond by']):
                action_items.append(sentence.strip())

        return action_items[:3]

    def _extract_letter_signature(self, lines: List[str]) -> Optional[str]:
        for i, line in enumerate(lines):
            if any(sig in line.lower() for sig in ['sincerely', 'yours truly', 'regards']):
                if i + 1 < len(lines):
                    return lines[i + 1].strip()
        return None

    def _extract_letter_attachments(self, text: str) -> List[str]:
        return re.findall(r'attachment[:]?\s*(.+)', text, re.IGNORECASE)

In [ ]:
class FormExtractor:
    def extract_form_data(self, text: str, layout_data: Dict) -> Dict:
        lines = text.split('\n')

        return {
            'form_type': self._identify_form_type(text),
            'form_fields': self._extract_form_fields(lines),
            'completed_sections': self._identify_completed_sections(lines),
            'signatures': self._extract_signatures(text),
            'dates': extract_dates(text),  # USING SHARED HELPER
            'checkboxes': self._extract_checkboxes(lines),
            'required_fields': self._identify_required_fields(lines)
        }

    def _identify_form_type(self, text: str) -> str:
        text_lower = text.lower()
        form_indicators = {
            'application_form': ['application', 'apply for', 'candidate information'],
            'registration_form': ['registration', 'register for', 'sign up'],
            'survey_form': ['survey', 'questionnaire', 'feedback'],
            'medical_form': ['medical', 'health', 'patient', 'symptoms'],
            'legal_form': ['legal', 'agreement', 'contract', 'terms']
        }

        for form_type, indicators in form_indicators.items():
            if any(indicator in text_lower for indicator in indicators):
                return form_type

        return 'general_form'

    def _extract_form_fields(self, lines: List[str]) -> List[Dict]:
        fields = []
        field_patterns = [
            r'^(.*?)[:_]\s*(.*)$',
            r'^(.*?)\s+-\s+(.*)$',
        ]

        for line in lines:
            for pattern in field_patterns:
                match = re.match(pattern, line.strip())
                if match:
                    field_name = match.group(1).strip()
                    field_value = match.group(2).strip()

                    if field_value and field_value not in ['', '_', '__']:
                        fields.append({
                            'field': field_name,
                            'value': field_value,
                            'field_type': self._classify_field_type(field_name)
                        })

        return fields

    def _classify_field_type(self, field_name: str) -> str:
        field_lower = field_name.lower()

        if any(word in field_lower for word in ['name', 'first name', 'last name']):
            return 'name'
        elif any(word in field_lower for word in ['email', 'e-mail']):
            return 'email'
        elif any(word in field_lower for word in ['phone', 'telephone', 'mobile']):
            return 'phone'
        elif any(word in field_lower for word in ['date', 'dob', 'birth']):
            return 'date'
        elif any(word in field_lower for word in ['address', 'street', 'city']):
            return 'address'
        elif any(word in field_lower for word in ['signature']):
            return 'signature'
        else:
            return 'text'

    def _identify_completed_sections(self, lines: List[str]) -> List[str]:
        sections = []
        current_section = None

        for line in lines:
            # Look for section headers (often in all caps or followed by colon)
            if (line.isupper() or line.endswith(':')) and len(line.strip()) > 3:
                current_section = line.strip().rstrip(':')

            # If we have a section and find form fields with values, mark as completed
            if current_section and any(':' in line and len(line.split(':', 1)[1].strip()) > 0 for line in lines):
                if current_section not in sections:
                    sections.append(current_section)

        return sections

    def _extract_signatures(self, text: str) -> List[str]:
        signature_indicators = ['signature', 'signed', 'approved by']
        signatures = []

        lines = text.split('\n')
        for i, line in enumerate(lines):
            if any(indicator in line.lower() for indicator in signature_indicators):
                # Look for signature in next lines
                for j in range(i+1, min(i+4, len(lines))):
                    if lines[j].strip() and not any(indicator in lines[j].lower() for indicator in signature_indicators):
                        signatures.append(lines[j].strip())
                        break

        return signatures

    def _extract_checkboxes(self, lines: List[str]) -> List[Dict]:
        checkboxes = []
        checkbox_patterns = [r'□', r'☐', r'\[\s*\]', r'\(\s*\)']  # Unchecked
        checked_patterns = [r'■', r'☑', r'✓', r'\[\s*X\s*\]', r'\(\s*X\s*\)']  # Checked

        for line in lines:
            for pattern in checked_patterns:
                if re.search(pattern, line):
                    checkboxes.append({
                        'field': line.strip(),
                        'checked': True
                    })
                    break
            else:
                for pattern in checkbox_patterns:
                    if re.search(pattern, line):
                        checkboxes.append({
                            'field': line.strip(),
                            'checked': False
                        })
                        break

        return checkboxes

    def _identify_required_fields(self, lines: List[str]) -> List[str]:
        required_fields = []
        required_indicators = ['*', 'required', 'mandatory']

        for line in lines:
            if any(indicator in line.lower() for indicator in required_indicators):
                # Extract field name before the indicator
                for indicator in required_indicators:
                    if indicator in line.lower():
                        field_name = line.split(indicator)[0].strip()
                        if field_name:
                            required_fields.append(field_name)
                        break

        return required_fields

In [ ]:
def test_all_extractors():
    """Test the structured data extraction for all document types"""
    extractor = StructuredDataExtractor()

    # Sample test data
    test_cases = {
        'invoice': "Invoice #INV-123\nFrom: Tech Supplies Inc.\nDate: 2024-01-15\nTotal: $1,500.00",
        'resume': "John Doe\nEmail: john@email.com\nPhone: 123-456-7890\nExperience: 5 years Python developer",
        'news_article': "Breaking News: Major Event Happens\nBy John Reporter\nPublished: January 15, 2024",
        'email': "From: sender@company.com\nTo: recipient@company.com\nSubject: Meeting Update\nDate: 2024-01-15",
        'letter': "Dear Mr. Smith,\nThis letter is regarding your application...\nSincerely,\nJohn Doe",
        'form': "Name: John Doe\nDate of Birth: 1990-01-01\nSignature: ________"
    }

    results = {}
    for doc_type, text in test_cases.items():
        print(f"\n🔍 Testing {doc_type} extraction...")
        result = extractor.extract_structured_data(doc_type, text)
        results[doc_type] = result
        print(f"✅ Extracted {len(result)} data points")

    return results

# Initialize and test
if __name__ == "__main__":
    print("🚀 Testing Structured Data Extraction for All 6 Document Types...")
    test_results = test_all_extractors()
    print(f"\n🎉 Successfully tested extraction for {len(test_results)} document types!")

🚀 Testing Structured Data Extraction for All 6 Document Types...

🔍 Testing invoice extraction...
✅ Extracted 11 data points

🔍 Testing resume extraction...
✅ Extracted 7 data points

🔍 Testing news_article extraction...
✅ Extracted 9 data points

🔍 Testing email extraction...
✅ Extracted 5 data points

🔍 Testing letter extraction...
✅ Extracted 4 data points

🔍 Testing form extraction...
✅ Extracted 7 data points

🎉 Successfully tested extraction for 6 document types!


In [ ]:
class DocuMindProductionPipeline:
    """Complete production pipeline integrating classification + extraction + summarization"""

    def __init__(self, model_path, processor, label_list):
        self.predictor = DocuMindPredictor(model_path, processor, label_list)
        self.data_extractor = StructuredDataExtractor()
        self.summarizer = DocumentSummarizer()

    def process_document(self, image_path):
        """Complete document processing: Classification → Extraction → Summarization"""
        import time
        start_time = time.time()

        try:
            # 1. CLASSIFICATION
            print("🔍 Classifying document...")
            classification_result = self.predictor.predict(image_path)
            doc_type = classification_result['document_type']

            # 2. OCR TEXT EXTRACTION
            print("📝 Extracting text with OCR...")
            image = Image.open(image_path)
            text = pytesseract.image_to_string(image)

            # 3. STRUCTURED DATA EXTRACTION
            print("🏗️ Extracting structured data...")
            extracted_data = self.data_extractor.extract_structured_data(doc_type, text)

            # 4. DOCUMENT SUMMARIZATION
            print("📊 Generating summary...")
            summary = self.summarizer.generate_summary(doc_type, extracted_data, text)

            # 5. CALCULATE PROCESSING METRICS
            processing_time = round(time.time() - start_time, 2)

            # 6. COMPILE FINAL RESULTS
            final_result = {
                'classification': classification_result,
                'extraction': extracted_data,
                'summary': summary,
                'metadata': {
                    'processing_time': processing_time,
                    'file_name': os.path.basename(image_path),
                    'text_length': len(text),
                    'document_type': doc_type,
                    'confidence': classification_result['confidence']
                }
            }

            print(f"✅ Processing completed in {processing_time}s")
            return final_result

        except Exception as e:
            print(f"❌ Error processing document: {e}")
            return {
                'error': str(e),
                'file_name': os.path.basename(image_path)
            }

In [ ]:
class EnhancedDocumentSummarizer:
    """Enhanced summarization with actionable insights"""

    def generate_summary(self, doc_type: str, extracted_data: Dict, raw_text: str) -> Dict:
        """Generate comprehensive summary with insights"""
        base_summary = self._generate_base_summary(doc_type, extracted_data, raw_text)
        insights = self._generate_insights(doc_type, extracted_data)
        actions = self._generate_actions(doc_type, extracted_data)

        return {
            'summary': base_summary,
            'key_insights': insights,
            'recommended_actions': actions,
            'urgency_level': self._assess_urgency(doc_type, extracted_data)
        }

    def _generate_base_summary(self, doc_type: str, data: Dict, text: str) -> str:
        """Generate the main summary text"""
        if doc_type == 'invoice':
            return self._summarize_invoice(data)
        elif doc_type == 'resume':
            return self._summarize_resume(data)
        elif doc_type == 'news_article':
            return self._summarize_news(data)
        elif doc_type == 'email':
            return self._summarize_email(data)
        elif doc_type == 'letter':
            return self._summarize_letter(data)
        elif doc_type == 'form':
            return self._summarize_form(data)
        else:
            return f"Processed {doc_type} document with key information extracted."

    def _summarize_invoice(self, data: Dict) -> str:
        amounts = data.get('total_amount')
        vendor = data.get('vendor', 'Unknown Vendor')
        due_date = data.get('due_date')

        summary_parts = [f"Invoice from {vendor}"]

        if amounts:
            summary_parts.append(f"Total: ${amounts:,.2f}")
        if due_date:
            summary_parts.append(f"Due: {due_date}")

        line_items = data.get('line_items', [])
        if line_items:
            summary_parts.append(f"Contains {len(line_items)} line items")

        return " | ".join(summary_parts)

    def _summarize_resume(self, data: Dict) -> str:
        personal = data.get('personal_info', {})
        experience = data.get('experience', [])
        skills = data.get('skills', {})

        summary_parts = []

        if personal.get('name'):
            summary_parts.append(f"Candidate: {personal['name']}")

        summary_parts.append(f"Experience: {len(experience)} positions")

        # Count total skills
        total_skills = sum(len(skill_list) for skill_list in skills.values())
        if total_skills:
            summary_parts.append(f"Skills: {total_skills} competencies")

        return " | ".join(summary_parts)

    def _generate_insights(self, doc_type: str, data: Dict) -> List[str]:
        """Generate actionable insights"""
        insights = []

        if doc_type == 'invoice':
            if data.get('due_date'):
                insights.append(f"Payment due on {data['due_date']}")
            if data.get('total_amount', 0) > 10000:
                insights.append("Large invoice amount - may require approval")

        elif doc_type == 'resume':
            exp_count = len(data.get('experience', []))
            if exp_count >= 5:
                insights.append("Senior-level candidate with extensive experience")
            elif exp_count <= 2:
                insights.append("Early-career candidate - consider for junior roles")

        elif doc_type == 'email':
            if data.get('urgency') == 'high':
                insights.append("Urgent email requiring immediate attention")

        return insights

    def _generate_actions(self, doc_type: str, data: Dict) -> List[str]:
        """Generate recommended actions"""
        actions = []

        if doc_type == 'invoice':
            actions.append("Process payment according to terms")
            actions.append("File in accounting system")

        elif doc_type == 'resume':
            actions.append("Add to candidate database")
            actions.append("Schedule screening interview")

        elif doc_type == 'email':
            actions.append("Respond to sender")
            if data.get('attachments'):
                actions.append("Review attachments")

        return actions

    def _assess_urgency(self, doc_type: str, data: Dict) -> str:
        """Assess urgency level"""
        if doc_type == 'invoice':
            # Check if due date is soon
            due_date = data.get('due_date')
            if due_date:
                # Simple urgency check (you could add actual date comparison)
                return "high" if "urgent" in str(data).lower() else "medium"
        elif doc_type == 'email':
            return data.get('urgency', 'low')

        return "low"